In [3]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import error_inject_layer
import error_inject_optimizer
from tensorflow.python.util import deprecation

# smash warnings at beginning
deprecation._PRINT_DEPRECATION_WARNINGS = False


In [4]:
def pack_features_vector(features, labels):
  """Pack the features into a single array."""
  features = tf.stack(list(features.values()), axis=1)
  return features, labels

In [5]:
# grab iris dataset from cloud
# train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
# train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
#                                            origin=train_dataset_url)
# test_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"
# test_fp = tf.keras.utils.get_file(fname=os.path.basename(test_url),
#                                   origin=test_url)
# print("Local copy of the dataset file: {}".format(train_dataset_fp))

In [7]:
train_dataset_fp = "/home/derek/.keras/datasets/iris_training.csv"

test_fp = "/home/derek/.keras/datasets/iris_test.csv"

column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
feature_names = column_names[:-1]
label_name = column_names[-1]
class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']


batch_size = 32
train_dataset = tf.data.experimental.make_csv_dataset(
    train_dataset_fp,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)

test_dataset = tf.data.experimental.make_csv_dataset(
    test_fp,
    batch_size,
    column_names=column_names,
    label_name='species',
    num_epochs=1,
    shuffle=False)



In [8]:
#print dataset info
print("\n\n\n\n\n")
print("DATASET INFO:")
print(type(train_dataset))
print("Features: {}".format(feature_names))
print("Label: {}".format(label_name))

# This gets the next batch of data, randomly pulled from the training dataset
features, labels = next(iter(train_dataset))
print("FEATURES:")
print(features)
print(len(features))
print("LABELS:")
print(labels)







DATASET INFO:
<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>
Features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
Label: species
FEATURES:
OrderedDict([('sepal_length', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([4.4, 6.2, 5.9, 6. , 6. , 5. , 4.6, 5.1, 6.4, 4.6, 4.8, 5.5, 5.7,
       7.7, 4.7, 4.9, 6.5, 5.1, 5.1, 4.4, 6.3, 7.7, 4.9, 5.6, 5.5, 6.9,
       6.1, 5.8, 5. , 7.4, 4.9, 5.6], dtype=float32)>), ('sepal_width', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([3.2, 3.4, 3.2, 2.7, 3. , 3.5, 3.6, 3.8, 3.2, 3.4, 3. , 2.6, 4.4,
       3.8, 3.2, 3.1, 3. , 3.8, 3.5, 2.9, 2.5, 3. , 2.5, 2.9, 2.4, 3.1,
       2.9, 2.7, 3.2, 2.8, 3.1, 2.5], dtype=float32)>), ('petal_length', <tf.Tensor: shape=(32,), dtype=float32, numpy=
array([1.3, 5.4, 4.8, 5.1, 4.8, 1.6, 1. , 1.6, 5.3, 1.4, 1.4, 4.4, 1.5,
       6.7, 1.3, 1.5, 5.5, 1.5, 1.4, 1.4, 5. , 6.1, 4.5, 3.6, 3.8, 5.1,
       4.7, 5.1, 1.2, 6.1, 1.5, 3.9], dtype=float32)>), ('petal_width

In [9]:
# creates map dataset
test_dataset = test_dataset.map(pack_features_vector)
train_dataset = train_dataset.map(pack_features_vector)


In [10]:
features, labels = next(iter(train_dataset))
print("FEATURES, FIRST FIVE:")
print(features[:5])
print("labels")
print(labels[:5])
print(train_dataset)


# note, difficult to convert tensor to float and back again
print("features type: ", type(features))
print("features data type", type(features[0]))
print(features[0])
fl1 = features[0][0]
print(fl1)

FEATURES, FIRST FIVE:
tf.Tensor(
[[4.7 3.2 1.3 0.2]
 [6.1 3.  4.9 1.8]
 [6.9 3.1 5.1 2.3]
 [6.3 3.4 5.6 2.4]
 [5.  3.3 1.4 0.2]], shape=(5, 4), dtype=float32)
labels
tf.Tensor([0 2 2 2 0], shape=(5,), dtype=int32)
<MapDataset shapes: ((None, 4), (None,)), types: (tf.float32, tf.int32)>
features type:  <class 'tensorflow.python.framework.ops.EagerTensor'>
features data type <class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([4.7 3.2 1.3 0.2], shape=(4,), dtype=float32)
tf.Tensor(4.7, shape=(), dtype=float32)


In [11]:
# Define the model
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),
  error_inject_layer.DenseErrorLayer(5, activation=tf.nn.relu, error_rate=0.9),  # input shape required
  # error_inject_layer.DenseErrorLayer(6, activation=tf.nn.relu),
  tf.keras.layers.Dense(3)
])


TRAINING
INFERENCE


In [12]:
predictions = model(features)
print("\n\n")
print("PREDICTIONS")
print(predictions[:5])

INFERENCE



PREDICTIONS
tf.Tensor(
[[ 0.11817942  0.6226523   0.23269494]
 [ 0.14466292  0.4346968   0.19464624]
 [-0.07873648 -0.13030834 -0.0998444 ]
 [ 0.29100406  0.4452667   0.286919  ]
 [ 0.11831926  0.6268151   0.23369294]], shape=(5, 3), dtype=float32)


In [13]:
# convert to probability
tf.nn.softmax(predictions[:5])
print(tf.nn.softmax(predictions[:5]))

tf.Tensor(
[[0.26472938 0.43842164 0.296849  ]
 [0.29518324 0.39450434 0.31031242]
 [0.34143108 0.32426918 0.3342997 ]
 [0.3161835  0.36892205 0.3148945 ]
 [0.26419517 0.43930057 0.29650432]], shape=(5, 3), dtype=float32)


In [14]:
# predict class 
print("Prediction: {}".format(tf.argmax(predictions, axis=1)))
print("    Labels: {}".format(labels))

Prediction: [1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1]
    Labels: [0 2 2 2 0 0 2 2 0 0 0 2 0 2 0 1 2 2 0 1 2 0 2 1 0 0 1 0 2 1 0 2]


In [15]:
# define loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss(model, x, y):
  # print("call")
  y_hat = model(x, training=True)
  print("Y:")
  print(y)
  print("Y_HAT:")
  print(y_hat)

  return loss_object(y_true=y, y_pred=y_hat)

l = loss(model, features, labels)
print("Loss test: {}".format(l))

# define gradient
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)


TRAINING
Y:
tf.Tensor([0 2 2 2 0 0 2 2 0 0 0 2 0 2 0 1 2 2 0 1 2 0 2 1 0 0 1 0 2 1 0 2], shape=(32,), dtype=int32)
Y_HAT:
tf.Tensor(
[[ 0.11817942  0.6226523   0.23269494]
 [ 0.14466292  0.4346968   0.19464624]
 [-0.07873648 -0.13030834 -0.0998444 ]
 [ 0.29100406  0.4452667   0.286919  ]
 [ 0.11831926  0.6268151   0.23369294]
 [ 0.08580699  0.5047006   0.1800504 ]
 [ 0.18326971  0.36994857  0.19973697]
 [ 0.22022462  0.82393396  0.34404644]
 [ 0.1104206   0.6135557   0.22412163]
 [ 0.07472987  0.44545847  0.15805395]
 [ 0.10516592  0.5908983   0.21483567]
 [ 0.2499798   0.4700879   0.27239516]
 [ 0.1367069   0.72852224  0.27091655]
 [ 0.03392694  0.3597755   0.10498507]
 [ 0.19871688  0.9721219   0.3754833 ]
 [ 0.15604827  0.5681984   0.23907171]
 [ 0.2228446   0.53584814  0.26953667]
 [ 0.11130904  0.21752153  0.1147559 ]
 [ 0.15104382  0.79873264  0.29802236]
 [ 0.12165301  0.65531754  0.24256419]
 [ 0.12778655  0.02030786  0.07156854]
 [ 0.13614795  0.6886275   0.26202223]
 [ 0.0344

In [16]:
optimizer = error_inject_optimizer.SGDErrorInject(learning_rate=0.01)

loss_value, grads = grad(model, features, labels)

print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.trainable_variables))

print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
                                          loss(model, features, labels).numpy()))


TRAINING
Y:
tf.Tensor([0 2 2 2 0 0 2 2 0 0 0 2 0 2 0 1 2 2 0 1 2 0 2 1 0 0 1 0 2 1 0 2], shape=(32,), dtype=int32)
Y_HAT:
tf.Tensor(
[[ 0.11817942  0.6226523   0.23269494]
 [ 0.14466292  0.4346968   0.19464624]
 [-0.07873648 -0.13030834 -0.0998444 ]
 [ 0.29100406  0.4452667   0.286919  ]
 [ 0.11831926  0.6268151   0.23369294]
 [ 0.08580699  0.5047006   0.1800504 ]
 [ 0.18326971  0.36994857  0.19973697]
 [ 0.22022462  0.82393396  0.34404644]
 [ 0.1104206   0.6135557   0.22412163]
 [ 0.07472987  0.44545847  0.15805395]
 [ 0.10516592  0.5908983   0.21483567]
 [ 0.2499798   0.4700879   0.27239516]
 [ 0.1367069   0.72852224  0.27091655]
 [ 0.03392694  0.3597755   0.10498507]
 [ 0.19871688  0.9721219   0.3754833 ]
 [ 0.15604827  0.5681984   0.23907171]
 [ 0.2228446   0.53584814  0.26953667]
 [ 0.11130904  0.21752153  0.1147559 ]
 [ 0.15104382  0.79873264  0.29802236]
 [ 0.12165301  0.65531754  0.24256419]
 [ 0.12778655  0.02030786  0.07156854]
 [ 0.13614795  0.6886275   0.26202223]
 [ 0.0344

In [18]:
train_loss_results = []
train_accuracy_results = []

num_epochs = 500

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  # Training loop - using batches of 32
  for x, y in train_dataset:
    # Optimize the model
    loss_value, grads = grad(model, x, y)
    # print("LOSS VALUE")
    # print(loss_value)
    # print("GRADS")
    # print(grads)
    print("APPY GRADIENTS")
    # print(model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg(loss_value)  # Add current batch loss
    # Compare predicted label to actual label
    epoch_accuracy(y, model(x))

  # End epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  if epoch % 1 == 0:
    print("RESULTS")
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))
